In [56]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample


In [5]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [161]:
def createData():
    df = readCsv('1_Cov.csv')
    meanPositive = readCsv('1_m_1.csv').dropna(axis=1, how='all').as_matrix()
    meanNegative = readCsv('1_m_0.csv').dropna(axis=1, how='all').as_matrix()
    meanPositive = meanPositive[0,:]
    meanNegative = meanNegative[0,:]
    
    cov = df.dropna(axis=1, how='all').as_matrix()
    
    distributionPositive = np.random.multivariate_normal(meanPositive, cov, 2000)
    distributionNegative = np.random.multivariate_normal(meanNegative, cov, 2000)
    
    distributionPositive = np.c_[distributionPositive, np.ones(distributionPositive.shape[0])]
    distributionNegative = np.c_[distributionNegative, -1*np.ones(distributionNegative.shape[0])]
#     print('Positive', distributionPositive)
#     print('Negative', distributionNegative)
    distribution = np.concatenate((distributionPositive, distributionNegative), axis=0)
    np.random.shuffle(distribution)
    distributionTest = distribution[0:1200]
    distributionTrain = distribution[1200:] 
    print(distribution)
    print(distributionTest)
    print(distributionTrain)
    print('original', distribution.shape)
    print('train', distributionTrain.shape)
    print('test', distributionTest.shape)
    return (distribution, distributionTest, distributionTrain)


In [162]:
def initializeDS1():
    fullDistribution, distributionTest, distributionTrain = createData()
    toCsv('1_train.csv', distributionTrain)
    toCsv('1_test.csv', distributionTest)
    toCsv('1.csv', fullDistribution)
initializeDS1()


[[ 4.98537155  5.06674229  4.21884673 ...,  6.54312634  6.37693658 -1.        ]
 [-0.88802452  2.45509744 -1.3399656  ...,  0.81971277  1.43013613  1.        ]
 [ 3.20384686  0.02048973  4.02284028 ..., -2.31229783  1.97465314 -1.        ]
 ..., 
 [ 0.25039995 -2.07577561  0.6402229  ...,  1.91321538  2.77713604 -1.        ]
 [ 4.31805663  1.89805318  4.2003596  ...,  2.83153413  3.44798384  1.        ]
 [-0.64865468 -3.69737769 -2.83872472 ..., -1.89019625 -1.95710203  1.        ]]
[[  4.98537155e+00   5.06674229e+00   4.21884673e+00 ...,   6.54312634e+00
    6.37693658e+00  -1.00000000e+00]
 [ -8.88024522e-01   2.45509744e+00  -1.33996560e+00 ...,   8.19712775e-01
    1.43013613e+00   1.00000000e+00]
 [  3.20384686e+00   2.04897274e-02   4.02284028e+00 ...,  -2.31229783e+00
    1.97465314e+00  -1.00000000e+00]
 ..., 
 [  2.69440765e+00   1.72182849e+00  -1.95739679e-03 ...,   9.49108129e-01
    1.57897684e+00   1.00000000e+00]
 [ -4.27267568e+00  -2.93072414e+00  -3.44201573e+00 ...,

In [124]:
def visualizeStuff():
    mean = [10, 10, 10]
    cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]] 
    x, y, z = np.random.multivariate_normal(mean, cov, 10000).T

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(111, projection='3d')

    plt.scatter(x, y, z, c='red')
    plt.axis('equal')
    plt.show()
# visualizeStuff()

In [141]:
def toCsv(fileName, npArray):
    
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')
